In [1]:
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn
import xarray as xr
import geopandas as gpd
import cartopy.crs as ccrs
from bokeh.models import HoverTool
import holoviews as hv
from holoviews import streams

In [2]:
pn.extension(design="material", sizing_mode="stretch_width")

In [3]:
@pn.cache
def get_data():
    ds1= xr.open_dataset("data/LENS_precomputed_terms_combined.nc")
    gdf_regions = gpd.read_file('data/rainfall_regions.geojson',).set_index('index')
    ds1 = ds1.assign_coords(region_id = gdf_regions.index.values)
    ds1 = xr.concat([ds1,ds1.sum('source').assign_coords(source='all')], dim='source') 
    
    # gdf_regions.index = [K39_regions[k] for k in gdf_regions.index]
    

    return ds1, gdf_regions

ds1, gdf_regions = get_data()
# data.tail()

/tmp/ipykernel_3063846/2795227684.py:6: FutureWarning: In a future version of xarray the default value for data_vars will change from data_vars='all' to data_vars=None. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set data_vars explicitly.
  ds1 = xr.concat([ds1,ds1.sum('source').assign_coords(source='all')], dim='source')


In [6]:
# model='CESM2'
# season='DJF'
# term='bias'
# source='dyn'
# vmax=50
#     # """Plots the rolling average and the outliers"""
# ds_sel = (
# ds1
# .sel(model=model, season=season, term=term, source=source)
# .sum("synoptic_bin") / 0.05 * 100
# )

# ds_sel.individual_term.to_series()

In [19]:
def get_plot(model='CESM2', season='DJF', term='bias', source='dyn', vmax=50):
    """Plots the rolling average and the outliers"""
    ds_sel = (
        ds1
        .sel(model=model, season=season, term=term, source=source)
        .sum("synoptic_bin") / 0.05 * 100
    )

    gdf = gdf_regions.copy()
    gdf[term] = ds_sel.individual_term.to_series()
    # hover = HoverTool(tooltips=tooltips)
    # hover = HoverTool(tooltips = [('@index', '@term')])
    poly_plot = gdf.hvplot.polygons( crs=ccrs.PlateCarree(), color=term)\
        .opts(height=500, 
              width=500, 
              # tools=['hover'],
              tools=['tap', 'hover'],  # Ensure tap is here
              # selection_mode='single',   # Or 'multiple'
              cmap = 'RdBu',
              color=term, 
              colorbar=True, 
              clim=(-vmax, vmax),
              projection=ccrs.PlateCarree(),
              axiswise=False, 
              shared_axes=True
             )#+gv.feature.ocean
    selection = streams.Selection1D(source=poly_plot)
    @pn.depends(selection.param.index)
    def update_secondary_plot(index):
        if not index:
            return "### Please select a region"
        # print(index)
        regions = gdf.iloc[index].index
        # region_names='  \n'+'  \n'.join(regions)
        # data=gdf.loc[index].index
        ds_sel_barplot = (
                ds1
                .sel(model=model, season=season, term=term,  region_id=regions)
                .sum("synoptic_bin") / 0.05 * 100
            ).individual_term
        series =  ds_sel_barplot.to_series().unstack().T#
        # series.index = series.index.reorder_levels([1,0])

        bars = series.hvplot.barh(title=term)
        return bars.opts(height=500, 
                         width=200)*hv.HLine(0).opts(color='k')#f"### You clicked on:  \n{region_names}"

    
    layout = pn.Row(poly_plot, update_secondary_plot)
    return layout

In [5]:
model_widget = pn.widgets.RadioButtonGroup(
    description="Model selection",
    name="Model",
    options=list(ds1.model.values),
)
season_widget = pn.widgets.RadioButtonGroup(
    description="Season selection",
    name="Season",
    options=list(ds1.season.values),
)
term_widget = pn.widgets.RadioButtonGroup(
    description="Term selection",
    name="Term",
    options=['bias','trend'],
)
source_widget = pn.widgets.RadioButtonGroup(
    description="Source selection",
    name="Source",
    options=list(ds1.source.values),
)
vmax_widget = pn.widgets.IntSlider(name="Colormap max value", value=50, start=1, end=150)

In [6]:
bound_plot = pn.bind(
    get_plot, model=model_widget, season=season_widget, term=term_widget, source=source_widget, vmax=vmax_widget
)

In [7]:
app = pn.template.MaterialTemplate(
    site="Panel",
    title="CMIP European rainfall decomposition",
    sidebar=[term_widget, source_widget, season_widget,model_widget, vmax_widget],
    main=[bound_plot],
)

/Data/gfi/users/rogui7909/virtual_envs/ml-robin/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1601: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(


In [8]:
app

MaterialTemplate
    [js_area] HTML(None, design=<class 'panel.theme.materi..., height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-140516075600352] ParamFunction(function, _pane=Row, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [nav-140516074781200] RadioButtonGroup(description='Term selection', design=<class 'panel.theme.materi..., name='Term', options=['bias', 'trend'], sizing_mode='stretch_width', value='bias')
    [nav-140516337825360] RadioButtonGroup(description='Source selection', design=<class 'panel.theme.materi..., name='Source', options=[np.str_('dyn'), ...], sizing_mode='stretch_width', value=np.str_('dyn'))
    [nav-140516016478000] RadioButtonGroup(description='Season selection', design=<class 'panel.theme.materi..., name='Season', options=[np.str_('DJF'), ...], sizing_mode='stretch_width', value=np.str_('DJF'))
    [nav-140516341924544] RadioButtonGroup(description='Model selection', design=<class 'panel.theme.materi..., name='Model', options=[np.str_('CESM2'), ...], sizing_mode='stretch_width', value=np.str_('CESM2'))
    [nav-140516337059312] IntSlider(design=<class 'panel.theme.materi..., end=150, name='Colormap max value', sizing_mode='stretch_width', start=1, value=50)

In [9]:
# bound_plot

In [ ]:
app.servable()

MaterialTemplate
    [js_area] HTML(None, design=<class 'panel.theme.materi..., height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-140179945821568] ParamFunction(function, _pane=Row, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [nav-140179940659456] RadioButtonGroup(description='Term selection', design=<class 'panel.theme.materi..., name='Term', options=['bias', 'trend'], sizing_mode='stretch_width', value='bias')
    [nav-140180013019216] RadioButtonGroup(description='Source selection', design=<class 'panel.theme.materi..., name='Source', options=[np.str_('dyn'), ...], sizing_mode='stretch_width', value=np.str_('dyn'))
    [nav-140180071939584] RadioButtonGroup(description='Season selection', design=<class 'panel.theme.materi..., name='Season', options=[np.str_('DJF'), ...], sizing_mode='stretch_width', value=np.str_('DJF'))
    [nav-140180358416960] RadioButtonGroup(description='Model selection', design=<class 'panel.theme.materi..., name='Model', options=[np.str_('CESM2'), ...], sizing_mode='stretch_width', value=np.str_('CESM2'))
    [nav-140179940899776] IntSlider(design=<class 'panel.theme.materi..., end=150, name='Colormap max value', sizing_mode='stretch_width', start=1, value=50)